### Daten laden und Zusammenführen
In diesem Schritt laden wir die Daten aus den Pickle-Dateien und führen die DataFrames zusammen.


In [1]:
import pandas as pd

# Daten laden
df_occupation = pd.read_pickle('./pickle/naics_occupation.pkl')
df_pattern = pd.read_pickle('./pickle/naics_pattern.pkl')

# Daten zusammenführen
merged_df = pd.merge(df_occupation, df_pattern, on=['FIPS', 'naics'], how='inner')


### Überprüfung der Überschneidungen:

In [2]:
merged_df.head()

,FIPS,State_GEOID_x,naics,NAICS_TITLE,emp_total_county_naics,OCC_CODE,OCC_TITLE,emp_occupation,State_GEOID_y,County_GEOID,...,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,n1000_1,n1000_2,n1000_3,n1000_4
0,13073,13,2373,"Highway, Street, and Bridge Construction",27,11-3051,Industrial Production Managers,0.022497,13,073,...,N,N,N,N,N,N,N,N,N,N
1,13073,13,2381,"Foundation, Structure, and Building Exterior C...",231,11-3051,Industrial Production Managers,0.067490,13,073,...,4,N,N,N,N,N,N,N,N,N
2,13073,13,2382,Building Equipment Contractors,868,11-3051,Industrial Production Managers,0.134981,13,073,...,10,N,N,N,N,N,N,N,N,N
3,13073,13,2383,Building Finishing Contractors,281,11-3051,Industrial Production Managers,0.078739,13,073,...,6,N,N,N,N,N,N,N,N,N
4,13073,13,3211,Sawmills and Wood Preservation,116,11-3051,Industrial Production Managers,0.978609,13,073,...,N,N,N,N,N,N,N,N,N,N


### Umwandlung von Zeichenfolgen in numerische Werte

In [3]:
# Funktion zur Umwandlung der Zeichenfolgen in numerische Werte
def convert_to_numeric(series):
    return series.replace('N', 0).astype(int)

# Umwandlung der Zeichenfolgen in den Unternehmensgrößenklassen in numerische Werte
size_columns = ['n<5', 'n5_9', 'n10_19', 'n20_49', 'n50_99', 'n100_249', 'n250_499', 'n500_999', 'n1000', 'n1000_1', 'n1000_2', 'n1000_3', 'n1000_4']
for col in size_columns:
    merged_df[col] = convert_to_numeric(merged_df[col])

### Festlegen der Aggregationsmethoden
Für jede numerische Spalte legen wir fest, wie die Werte aggregiert werden sollen. Summen werden für absolute Zahlen wie Beschäftigungszahlen und Unternehmensgrößenklassen berechnet.

In [4]:
# Festlegen der Aggregationsmethoden
agg_methods = {
    'emp_total_county_naics': 'sum',
    'emp_occupation': 'sum',
    'emp': 'sum',
    'qp1': 'sum',
    'ap': 'sum',
    'est': 'sum',
    'n<5': 'sum',
    'n5_9': 'sum',
    'n10_19': 'sum',
    'n20_49': 'sum',
    'n50_99': 'sum',
    'n100_249': 'sum',
    'n250_499': 'sum',
    'n500_999': 'sum',
    'n1000': 'sum',
    'n1000_1': 'sum',
    'n1000_2': 'sum',
    'n1000_3': 'sum',
    'n1000_4': 'sum'
}


### Aggregation der Daten
Die Daten werden nach dem FIPS-Code gruppiert und gemäß den festgelegten Aggregationsmethoden aggregiert.

In [5]:
# Aggregation der Daten nach FIPS
aggregated_df = merged_df.groupby(['FIPS', 'naics']).agg(agg_methods).reset_index()
aggregated_df.head()

# Pivot the aggregated dataframe to create a column for each NAICS in each county
pivot_df = aggregated_df.pivot_table(
    index='FIPS',
    columns='naics',
    values=list(agg_methods.keys()),
    aggfunc='sum'
)

# Flatten the multi-level columns
pivot_df.columns = ['_'.join(map(str, col)).strip() for col in pivot_df.columns.values]

# Reset index to make 'FIPS' a column again
pivot_df.reset_index(inplace=True)

### Zusätzliche Features erstellen

In [6]:
# Gesamte vierteljährliche Lohnsumme über alle NAICS-Codes hinweg
pivot_df['total_qp1'] = pivot_df.filter(like='qp1').sum(axis=1)

# Gesamtzahl der Beschäftigten über alle NAICS-Codes hinweg
pivot_df['total_emp'] = pivot_df.filter(like='emp').sum(axis=1)

# Industrievielfalt-Index (ein einfaches Beispiel basierend auf der Anzahl der unterschiedlichen NAICS-Codes)
pivot_df['industry_diversity_index'] = pivot_df.filter(like='emp').count(axis=1)

In [7]:
#Erstellen zusätzlicher Features wie Anteil der Beschäftigten pro NAICS-Code in jedem County
for col in list(agg_methods.keys()):
    if col in pivot_df.columns:
        pivot_df[f'{col}_percentage'] = pivot_df[col].div(pivot_df.groupby('FIPS')[col].transform('sum')) * 100
        
# Summieren der Unternehmensgrößenklassen für das gesamte County
county_size_features = pivot_df.groupby('FIPS').sum().reset_index()
county_size_features = county_size_features[['FIPS'] + [col for col in county_size_features.columns if any(size in col for size in size_columns)]]

# Umbenennen der Spalten, um die Aggregation widerzuspiegeln
county_size_features.columns = ['FIPS'] + [f'total_{col}' for col in county_size_features.columns if col != 'FIPS']

# Zusammenführen der zusätzlichen Features in den Pivot DataFrame
final_df = pd.merge(pivot_df, county_size_features, on='FIPS', how='left')

final_df.head()

,FIPS,ap_1133,ap_2111,ap_2121,ap_2122,ap_2123,ap_2131,ap_2211,ap_2212,ap_2362,...,total_n<5_4881,total_n<5_4882,total_n<5_5321,total_n<5_5413,total_n<5_5612,total_n<5_5613,total_n<5_5617,total_n<5_7139,total_n<5_8111,total_n<5_8113
0,1001,36504.0,NaN,NaN,NaN,102880.0,NaN,294156.0,NaN,75312.0,...,0.0,0.0,0.0,110.0,0.0,0.0,253.0,24.0,192.0,0.0
1,1003,NaN,NaN,NaN,NaN,25820.0,4446.0,292698.0,NaN,448260.0,...,120.0,0.0,56.0,1798.0,45.0,312.0,1364.0,208.0,1083.0,217.0
2,1005,36256.0,NaN,NaN,NaN,NaN,NaN,202374.0,NaN,NaN,...,0.0,0.0,0.0,132.0,0.0,0.0,44.0,0.0,112.0,0.0
3,1007,4224.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,65.0,0.0
4,1009,NaN,NaN,NaN,NaN,14860.0,NaN,NaN,NaN,32706.0,...,0.0,0.0,0.0,0.0,0.0,0.0,143.0,35.0,240.0,390.0


# Feature-Beschreibungen

## Originale Features

1. **FIPS**: Federal Information Processing Standards Code, der Landkreise und county-äquivalente Einheiten in den USA eindeutig identifiziert.
2. **naics_X_emp_total_county_naics**: Gesamtzahl der Beschäftigten in der angegebenen NAICS-Industrie innerhalb des Landkreises.
3. **naics_X_emp_occupation**: Anzahl der Beschäftigten in einem bestimmten Beruf innerhalb der angegebenen NAICS-Industrie im Landkreis.
4. **naics_X_emp**: Gesamtzahl der Beschäftigten in der angegebenen NAICS-Industrie.
5. **naics_X_qp1**: Gesamtbetrag der vierteljährlichen Lohnsumme für die angegebene NAICS-Industrie.
6. **naics_X_ap**: Gesamtbetrag der jährlichen Lohnsumme für die angegebene NAICS-Industrie.
7. **naics_X_est**: Gesamtzahl der Betriebe in der angegebenen NAICS-Industrie.
8. **naics_X_n<5**: Anzahl der Betriebe mit 1 bis 4 Beschäftigten in der angegebenen NAICS-Industrie.
9. **naics_X_n5_9**: Anzahl der Betriebe mit 5 bis 9 Beschäftigten in der angegebenen NAICS-Industrie.
10. **naics_X_n10_19**: Anzahl der Betriebe mit 10 bis 19 Beschäftigten in der angegebenen NAICS-Industrie.
11. **naics_X_n20_49**: Anzahl der Betriebe mit 20 bis 49 Beschäftigten in der angegebenen NAICS-Industrie.
12. **naics_X_n50_99**: Anzahl der Betriebe mit 50 bis 99 Beschäftigten in der angegebenen NAICS-Industrie.
13. **naics_X_n100_249**: Anzahl der Betriebe mit 100 bis 249 Beschäftigten in der angegebenen NAICS-Industrie.
14. **naics_X_n250_499**: Anzahl der Betriebe mit 250 bis 499 Beschäftigten in der angegebenen NAICS-Industrie.
15. **naics_X_n500_999**: Anzahl der Betriebe mit 500 bis 999 Beschäftigten in der angegebenen NAICS-Industrie.
16. **naics_X_n1000**: Anzahl der Betriebe mit 1000 oder mehr Beschäftigten in der angegebenen NAICS-Industrie.
17. **naics_X_n1000_1**: Anzahl der Betriebe mit 1000 bis 1499 Beschäftigten in der angegebenen NAICS-Industrie.
18. **naics_X_n1000_2**: Anzahl der Betriebe mit 1500 bis 2499 Beschäftigten in der angegebenen NAICS-Industrie.
19. **naics_X_n1000_3**: Anzahl der Betriebe mit 2500 bis 4999 Beschäftigten in der angegebenen NAICS-Industrie.
20. **naics_X_n1000_4**: Anzahl der Betriebe mit 5000 oder mehr Beschäftigten in der angegebenen NAICS-Industrie.
21. **naics_X_emp_percentage**: Prozentsatz der Gesamtbeschäftigung in der angegebenen NAICS-Industrie relativ zur Gesamtbeschäftigung im Landkreis.

## Zusätzliche Features für das gesamte County

1. **total_n<5**: Gesamtzahl der Betriebe mit 1 bis 4 Beschäftigten im gesamten Landkreis, unabhängig von der NAICS-Industrie.
2. **total_n5_9**: Gesamtzahl der Betriebe mit 5 bis 9 Beschäftigten im gesamten Landkreis, unabhängig von der NAICS-Industrie.
3. **total_n10_19**: Gesamtzahl der Betriebe mit 10 bis 19 Beschäftigten im gesamten Landkreis, unabhängig von der NAICS-Industrie.
4. **total_n20_49**: Gesamtzahl der Betriebe mit 20 bis 49 Beschäftigten im gesamten Landkreis, unabhängig von der NAICS-Industrie.
5. **total_n50_99**: Gesamtzahl der Betriebe mit 50 bis 99 Beschäftigten im gesamten Landkreis, unabhängig von der NAICS-Industrie.
6. **total_n100_249**: Gesamtzahl der Betriebe mit 100 bis 249 Beschäftigten im gesamten Landkreis, unabhängig von der NAICS-Industrie.
7. **total_n250_499**: Gesamtzahl der Betriebe mit 250 bis 499 Beschäftigten im gesamten Landkreis, unabhängig von der NAICS-Industrie.
8. **total_n500_999**: Gesamtzahl der Betriebe mit 500 bis 999 Beschäftigten im gesamten Landkreis, unabhängig von der NAICS-Industrie.
9. **total_n1000**: Gesamtzahl der Betriebe mit 1000 oder mehr Beschäftigten im gesamten Landkreis, unabhängig von der NAICS-Industrie.
10. **total_n1000_1**: Gesamtzahl der Betriebe mit 1000 bis 1499 Beschäftigten im gesamten Landkreis, unabhängig von der NAICS-Industrie.
11. **total_n1000_2**: Gesamtzahl der Betriebe mit 1500 bis 2499 Beschäftigten im gesamten Landkreis, unabhängig von der NAICS-Industrie.
12. **total_n1000_3**: Gesamtzahl der Betriebe mit 2500 bis 4999 Beschäftigten im gesamten Landkreis, unabhängig von der NAICS-Industrie.
13. **total_n1000_4**: Gesamtzahl der Betriebe mit 5000 oder mehr Beschäftigten im gesamten Landkreis, unabhängig von der NAICS-Industrie.
14. **total_qp1**: Gesamte vierteljährliche Lohnsumme über alle NAICS-Codes hinweg.
15. **total_emp**: Gesamtzahl der Beschäftigten über alle NAICS-Codes hinweg.
16. **industry_diversity_index**: Industrievielfalt-Index, basierend auf der Anzahl der unterschiedlichen NAICS-Codes.
"""

### Speichern als Pickle

In [8]:
final_df.to_pickle('./pickle/final_df.pkl')